<a href="https://colab.research.google.com/github/javgat/viu-algoritmos-de-optimizacion/blob/master/Seminario_Algoritmos_Javier_Gat%C3%B3n_Herguedas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Javier Gatón Herguedas<br>
Drive: https://colab.research.google.com/drive/1cZyXJfWB6tBkHP6kc7geVWn_gTe2e8iL?usp=sharing <br>
Url: https://github.com/javgat/viu-algoritmos-de-optimizacion<br>
Problema:
> 1. Sesiones de doblaje <br>

## Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

(*) La respuesta es obligatoria





                                        

In [1]:
# importo librerías que se utilizarán
import math
from typing import Set, List, FrozenSet
from itertools import combinations
import random
import numpy as np
import warnings

## Espacio de soluciones

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?




Respuesta:

Para calcular el tamaño del espacio de soluciones necesitamos utilizar los números de Bell, los cuales utilizan los números de Stirling de segundo tipo.

### Números de Stirling de Segundo Tipo

Los números de Stirling de segundo tipo son la cantidad de maneras que existen de dividir “n” elementos en “k” conjuntos no vacíos.

Se pueden definir de forma recursiva como:

$S(n, k) = k* S(n-1, k) + S(n-1, k-1)$

### Números de Bell

Los números de Bell permiten hallar cuantas formas de dividir un conjunto en subconjuntos no vacíos existen, lo cual es exactamente lo que queremos para la primera respuesta.

Se definen de la siguiente forma:

$ B$<sub>n</sub>$ = \sum_{k=1}^n S(n,k)$


### Números de Stirling de Segundo Tipo y Bell restringidos.

Estos números son como los números de Stirling pero donde los subconjuntos tienen un tamaño máximo m.

Sabemos su definición recursiva tal que:

$S(n+1,k)$<sub>≤m</sub>$ = k * S(n,k)$<sub>≤m</sub>$ + S(n,k−1)$<sub>≤m</sub>$ − \binom n m * S(n−m,k−1)$<sub>≤m</sub>

A partir de ellos se pueden calcular los números de Bell restringidos, que se definen de la siguiente forma:

$ B$<sub>n,≤m</sub>$ = \sum_{k=1}^n S(n,k)$<sub>≤m</sub>

### Respuesta

Sin tener en cuenta las restricciones, entendiendo las restricciones por "No es posible grabar más de 6 tomas por día", entonces hay $B$<sub>30</sub> posibilidades, siendo esto el número de Bell de 30, que vale **846749014511809332450147**.

Teniendo en cuenta la restricción de un máximo de 6 tomas por día, entonces podemos calcular las posibilidades existentes utilizando los números de Bell restringidos, entonces la solución es $B$<sub>30,≤6</sub> , que vale **726391948970868949621309**.

### Fuentes

Números de Stirling de segundo tipo y números de Bell: https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind

Números de Stirling de segundo tipo restringidos: https://math.stackexchange.com/questions/2302204/counting-set-partitions-of-1-2-n-into-exactly-k-non-empty-subsets-wi

Números de Bell restringidos: https://math.stackexchange.com/questions/3685565/set-partitioning-with-max-partition-size y https://arxiv.org/pdf/1707.08138.pdf

In [2]:
_STIRLING_DICT = {} # Diccionario que uso para almacenar valores de stirling y optimizar los cálculos

def stirling2(n, k):
  if n == k:
    return 1
  if k == 0 or n == 0:
    return 0
  if n < k:
    return 0
  if (n, k) in _STIRLING_DICT:
    return _STIRLING_DICT[(n, k)]
  val = k*stirling2(n-1, k) + stirling2(n-1, k-1)
  _STIRLING_DICT[(n, k)] = val
  return val

def bell_number(n: int) -> int:
  suma = 0
  for k in range(n+1):
    suma += stirling2(n, k)
  return suma

def bell_number_optimized(n: int) -> int:
  # Implementación inspirada en https://www.geeksforgeeks.org/bell-numbers-number-of-ways-to-partition-a-set/
  bell = [[0 for i in range(n+1)] for j in range(n+1)]
  bell[0][0] = 1
  for i in range(n):
    bell[i+1][0] = bell[i][i]
    for j in range(i+1):
      bell[i+1][j+1] = bell[i][j] + bell[i+1][j]
  return bell[n][0]

_STIRLING_RESTRICT_DICT = {}

def stirling2restrict(n, k, m):
  if n == k:
    return 1
  if k == 0 or n == 0:
    return 0
  if n < k:
    return 0
  if (n, k, m) in _STIRLING_RESTRICT_DICT:
    return _STIRLING_RESTRICT_DICT[(n, k, m)]
  val = k*stirling2restrict(n-1, k, m) + stirling2restrict(n-1, k-1, m) - math.comb(n-1, m) * stirling2restrict(n-1-m, k-1, m)
  _STIRLING_RESTRICT_DICT[(n, k, m)] = val
  return val

def bell_number_restrict(n: int, m: int) -> int:
  suma = 0
  for k in range(n+1):
    suma += stirling2restrict(n, k, m)
  return suma

N_ELEMS = 30
GROUPS_MAX_SIZE = 6

print(f"Número de Bell de {N_ELEMS} = {bell_number(N_ELEMS)}")
print(f"Número de Bell de {N_ELEMS} restringido a {GROUPS_MAX_SIZE} = {bell_number_restrict(N_ELEMS, GROUPS_MAX_SIZE)}")


Número de Bell de 30 = 846749014511809332450147
Número de Bell de 30 restringido a 6 = 726391948970868949621309


## Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)


Respuesta

### Datos de entrada

Para los datos de entrada del problema decido usar una lista de conjuntos de enteros. Cada conjunto representará una toma y su contenido son los actores necesarios para la misma.

En el ejercicio las tomas están indexadas desde el 1, pero en esta representación se indexarán desde el 0, por lo que la toma i de la hoja de cálculo será representada en la posición i-1 en la lista.

Posteriormente, a la hora de imprimir se puede añadir 1 al valor a imprimir para hacer la correlación.

Para los actores se mantendrá el valor que había en la hoja de cálculo, pues lo usaré como simplemente un identificador, no un índice, perfectamente podría ser una cadena.

### Datos de salida (solución)
Inicialmente para la solución pensé que la mejor estructura sería un conjunto de conjuntos de enteros que define qué tomas se graban el mismo día (cada subconjunto era un día). Esto en python se hace con un set de frozensets de index, porque no se puede hacer un set de sets (los sets no son hasheables, tienen que ser inmutables).

Sin embargo, para poder explorar las vecindades de la solución decido utilizar una lista de conjuntos, pues así la posición de cada conjunto (día) en la lista, aunque no afecte al coste, sirve para poder crear soluciones vecinas en algoritmos metaheurísticos de forma simple.

### Parsing de la hoja de cálculo

A continuación muestro como pasar de una cadena con los datos de la hoja de cálculo a la estructura de datos a utilizar.

In [3]:
_EXCEL_TABLE_STR = """
1 1 1 1 1 0 0 0 0 0
0	0	1	1	1	0	0	0	0	0
0	1	0	0	1	0	1	0	0	0
1	1	0	0	0	0	1	1	0	0
0	1	0	1	0	0	0	1	0	0
1	1	0	1	1	0	0	0	0	0
1	1	0	1	1	0	0	0	0	0
1	1	0	0	0	1	0	0	0	0
1	1	0	1	0	0	0	0	0	0
1	1	0	0	0	1	0	0	1	0
1	1	1	0	1	0	0	1	0	0
1	1	1	1	0	1	0	0	0	0
1	0	0	1	1	0	0	0	0	0
1	0	1	0	0	1	0	0	0	0
1	1	0	0	0	0	1	0	0	0
0	0	0	1	0	0	0	0	0	1
1	0	1	0	0	0	0	0	0	0
0	0	1	0	0	1	0	0	0	0
1	0	1	0	0	0	0	0	0	0
1	0	1	1	1	0	0	0	0	0
0	0	0	0	0	1	0	1	0	0
1	1	1	1	0	0	0	0	0	0
1	0	1	0	0	0	0	0	0	0
0	0	1	0	0	1	0	0	0	0
1	1	0	1	0	0	0	0	0	1
1	0	1	0	1	0	0	0	1	0
0	0	0	1	1	0	0	0	0	0
1	0	0	1	0	0	0	0	0	0
1	0	0	0	1	1	0	0	0	0
1	0	0	1	0	0	0	0	0	0"""

def _int_and_split(val: str) -> List[List[int]]:
  return list(map(int, val.split()))

def parse_table(table: str) -> List[Set[int]]:
  table_splitted = [row for row in table.split('\n') if row != '']
  table_list = list(map(_int_and_split, table_splitted))
  tomas = []
  for row in table_list:
    toma = {i+1 for i, val in enumerate(row) if val==1}
    tomas.append(toma)
  return tomas

TOMAS_ENUNCIADO = parse_table(_EXCEL_TABLE_STR)
display(TOMAS_ENUNCIADO)

def print_solucion(solucion: List[Set[int]]):
  # Imprime las soluciones indexada en 1
  printst = "Tomas: "
  for i, sol in enumerate(solucion):
    printst += f"Día {i+1}: "
    for j, s in enumerate(sol):
      if j == len(sol)-1:
        printst += f"{s+1}. "
      else:
        printst += f"{s+1}, "
  print(printst)

[{1, 2, 3, 4, 5},
 {3, 4, 5},
 {2, 5, 7},
 {1, 2, 7, 8},
 {2, 4, 8},
 {1, 2, 4, 5},
 {1, 2, 4, 5},
 {1, 2, 6},
 {1, 2, 4},
 {1, 2, 6, 9},
 {1, 2, 3, 5, 8},
 {1, 2, 3, 4, 6},
 {1, 4, 5},
 {1, 3, 6},
 {1, 2, 7},
 {4, 10},
 {1, 3},
 {3, 6},
 {1, 3},
 {1, 3, 4, 5},
 {6, 8},
 {1, 2, 3, 4},
 {1, 3},
 {3, 6},
 {1, 2, 4, 10},
 {1, 3, 5, 9},
 {4, 5},
 {1, 4},
 {1, 5, 6},
 {1, 4}]

## Según el modelo para el espacio de soluciones

(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Función objetivo: La función es el coste de los actores. El coste de cada actor es el número de días que va, y el coste de los actores es la suma del coste de cada actor.

¿Maximización o minimización?: Se trata de minimización, pues queremos minimizar el coste de los actores.

In [4]:
def coste_actores(sol: List[Set[int]], tomas: List[Set[int]]) -> int:
  if sol == None:
    return float("inf")
  suma = 0
  for dia in sol:
    suma += len({actor for toma_index in dia for actor in tomas[toma_index]})
  return suma

## Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [5]:
def partitions_restricted(collection: list, m: int):
  if len(collection) == 1:
    yield [ collection ]
    return
  first = collection[0]
  for smaller in partitions_restricted(collection[1:], m):
    # insert `first` in each of the subpartition's subsets
    for n, subset in enumerate(smaller):
      if len(subset) > m-1:
        continue
      yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
    # put `first` in its own subset
    yield [ [ first ] ] + smaller

def tomas_actores_fuerza_bruta(tomas: List[Set[int]], restriccion: int) -> List[Set[int]]:
  min_coste = float('inf')
  min_sol = None
  for part in partitions_restricted([i for i in range(len(tomas))], restriccion):
    sol = [set(sub) for sub in part]
    coste = coste_actores(sol, tomas)
    if coste < min_coste:
      min_coste = coste
      min_sol = sol
  return min_sol


#test_tomas = [{1,2}, {2,3}, {1,2,3}, {1,4}, {2,5,6}, {3,4}]
test_tomas = [{0}, {0,3}, {0,1,2}, {0,2,3}, {1,3}, {1,2}, {1,2,3}, {2}]
_sol = tomas_actores_fuerza_bruta(test_tomas, 3)
print_solucion(_sol)
print(f"Coste: {coste_actores(_sol, test_tomas)}")

Tomas: Día 1: 1, 2. Día 2: 3, 4, 5. Día 3: 6, 7, 8. 
Coste: 9


## Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Complejidad: $O(B$<sub>n,≤m</sub>$)$

La forma más directa y concreta de expresar la complejidad del algoritmo es utilizar el número de Bell restringido, pues su complejidad es ese número exactamente, que es $O(B$<sub>n,≤m</sub>$)$, siendo $B$<sub>n,≤m</sub> el número de Bell de n restringido a m.

Sin embargo, si se quisiera expresar de forma que cualquiera lo entendiera sin necesidad de conocer el número de Bell, se puede decir que su complejidad es $O(n^n - {n^n \over m})$.

Esta complejidad no sé como demostrarla, pues para hallar esta complejidad me he basado en la complejidad del número de Bell normal, que es $O(n^n)$, en mi intuición y en simples comprobaciones utilizando las funciones creadas al inicio del documento.

Por ejemplo, para el caso del enunciado, $B$<sub>30,≤6</sub> , he comprobado que $ B$<sub>30</sub>$ - (B$<sub>30</sub>$ / 6) = 0.971 * B$<sub>30,≤6</sub>

El orden de complejidad del número de Bell sin restricciones es $O(n^n)$, tal y como se puede deducir a partir de la siguiente relación que se muestra en la Wikipedia:

$ B$<sub>n+1</sub>$ = \sum_{k=0}^n \binom n k B$<sub>k</sub>

Que continúa con:

$ B$<sub>n+1</sub>$ \geq B$<sub>n</sub>$ + nB$<sub>n-1</sub>$ \geq nB$<sub>n-1</sub>

Lo cual implica:

$ B$<sub>n+1</sub>$ \geq n(n-2)(n-4) ... \geq (n/2)^{n/4} $

Lo cual es $O(n^n)$.

### Fuentes

https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind

https://cs.stackexchange.com/questions/92964/computational-complexity-of-bell-numbers

In [6]:
# Ejemplos mencionados:

b10 = bell_number_optimized(10)
b10_3 = bell_number_restrict(10, 3)
print((b10-b10/3)/b10_3)

b30 = bell_number_optimized(30)
b30_6 = bell_number_restrict(30, 6)
print((b30-b30/6)/b30_6)

b40 = bell_number_optimized(40)
b40_6 = bell_number_restrict(40, 6)
print((b40-b40/6)/b40_6)

b40_15 = bell_number_restrict(40, 15)
print((b40-b40/15)/b40_15)

1.2646667539038645
0.9714096910897057
1.1188390117690865
0.9333335261551129


## Diseña algoritmo mejorado

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta por qué crees que mejora el algoritmo por fuerza bruta

Respuesta

Inicialmente intento crear una solución voraz basándome en "sinergías" entre tomas, relación que hay entre tomas basadas en la cantidad de actores de una toma que son utilizados en otra toma. Concretamente: (num_actores_toma_i - num_actores_toma_i_no_toma_j) / num_actores_toma_i.

Genero una tabla de sinergías, la cual utilizo para hallar cuáles son las tomas más "amigables" (tomas con mayor media de sinergía con otras tomas), para poder asignarlas a días empezando por las más amigables. Primero selecciono la toma más amigable, y la pongo con las tomas con más sinergía con esa toma, estableciendo también cierta preferencia por aquellas que al mismo tiempo sean poco amigables. Probando diferentes valores he acabado determinando que la fórmula para escoger las tomas compañeras del día es la "amigabilidad" con la toma del día al cubo, dividido entre la amigabilidad general de la toma a añadir. Cuando he acabado con el día, hago lo mismo con la siguiente toma más "amigable" libre.

Este algoritmo voraz consigue resultados bastante buenos (coste 29 para el problema del enunciado), sin embargo creo poder mejorarlo usando metaheurísticas, aunque este algoritmo voraz puede servirme como base para las mismas.

Decido aplicar recocido simulado utilizando el algoritmo voraz como base. La vecina de cada iteración es la mejor vecina de entre un conjunto de múltiples vecinas generadas aleatoriamente basándome en 4 entornos. Los entornos son intercambiar 2 tomas de día, mover una toma a otro día, juntar 2 días que puedan ser juntados, e intercambiar 2 parejas de tomas de día, en días de 4 o más tomas.

Haciendo esto consigo mejorar levemente la optimización de mi algoritmo, alcanzando una puntuación de 27.

In [7]:
def sinergia_tomas(tomas: List[Set[int]]) -> List[List[float]]:
  sinergias = []
  for i, toma in enumerate(tomas):
    toma_sins = []
    for j, tcomp in enumerate(tomas):
      tactors = len(toma)
      sinscore = (tactors - len(toma-tcomp))/tactors
      toma_sins.append(sinscore)
    sinergias.append(toma_sins)
  return sinergias

def compact_solutions(solution: List[Set[int]], tomas: List[Set[int]], restriccion: int) -> None:
  #sinergias = np.array(sinergia_tomas(tomas))
  for i, dia in enumerate(solution):
    remove_days = []
    if len(dia) < restriccion:
      for j in range(i, len(solution)):
        if i == j:
          continue
        dia_propuesto = solution[j]
        if len(dia_propuesto)+len(dia) <= restriccion:
          dia.update(dia_propuesto)
          remove_days.append(j)
    for rd in reversed(sorted(remove_days)):
      solution.pop(rd)

def tomas_actores_greedy_sinergias(tomas: List[Set[int]], restriccion: int) -> List[Set[int]]:
  warnings.filterwarnings("ignore", message=".*Mean of empty slice.*", category=RuntimeWarning)
  sinergias = np.array(sinergia_tomas(tomas))
  np.fill_diagonal(sinergias, np.nan)
  used_ids = set()
  sol = []
  coef_popular = 3
  coef_individual = 1

  while len(used_ids) < len(tomas):
    mean_sinergias = np.nanmean(sinergias, axis=0)
    popular_indexes = (-mean_sinergias).argsort()
    fid = -1
    for ffid in popular_indexes:
      if ffid not in used_ids:
        fid = ffid
        break
    fid_friendliness = sinergias[fid]
    friends_relation = (fid_friendliness**coef_popular)/(mean_sinergias**coef_individual)
    id_tochoose = (-friends_relation).argsort()
    day = {fid}
    has_added = True
    cid_id = 0
    while len(day) < restriccion and cid_id < len(id_tochoose):
      has_added = False
      cid = id_tochoose[cid_id]
      if cid == fid or cid in used_ids:
        cid_id += 1
        continue
      mean_day = sinergias[cid][[did for did in day]].mean()
      if mean_day < mean_sinergias[cid]:
        cid_id += 1
        continue
      day.add(cid)
      used_ids.add(cid)
      cid_id = 0
    used_ids.add(fid)
    for did in day:
      sinergias[did, :] = np.nan
      sinergias[:, did] = np.nan
    sol.append(day)
  warnings.resetwarnings()
  compact_solutions(sol, tomas, restriccion)
  return sol

sg = tomas_actores_greedy_sinergias(TOMAS_ENUNCIADO, 6)
print_solucion(sg)
print(f"Coste: {coste_actores(sg, TOMAS_ENUNCIADO)}")


Tomas: Día 1: 1, 2, 6, 7, 20, 22. Día 2: 8, 9, 10, 12, 14, 25. Día 3: 3, 4, 5, 11, 15, 26. Día 4: 13, 16, 27, 28, 29, 30. Día 5: 17, 18, 19, 21, 23, 24. 
Coste: 29


In [11]:
def random_solution(tomas: List[Set[int]], restriccion: int) -> List[Set[int]]:
  n_tomas = len(tomas)
  tomas_order = list(range(n_tomas))
  random.shuffle(tomas_order)
  sol = []
  while tomas_order:
    day_size = random.randint(1, min(restriccion, len(tomas_order)))
    sol.append(set(tomas_order[:day_size]))
    tomas_order = tomas_order[day_size:]
  return sol

def swap_tomas(sol: List[Set[int]], id_0: int, id_1: int) -> List[Set[int]]:
  new_sol = []
  for dia in sol:
    new_dia = set(dia)
    if id_0 in dia and id_1 not in dia:
      new_dia.remove(id_0)
      new_dia.add(id_1)
    elif id_1 in dia and id_0 not in dia:
      new_dia.remove(id_1)
      new_dia.add(id_0)
    new_sol.append(new_dia)
  return new_sol

def move_to(sol: List[Set[int]], id_from: int, to_dia: int, restriccion: int) -> List[Set[int]]:
  # to_dia un dia existente o uno siguiente para en nuevo dia
  new_sol = []  
  for dia in sol:
    new_dia = set(dia)
    if id_from in dia:
      new_dia.remove(id_from)
    new_sol.append(set(new_dia))
  if len(new_sol) == to_dia:
    new_dia = {id_from}
    new_sol.append(new_dia)
  elif len(new_sol[to_dia]) >= restriccion:
    raise Exception(f"Illegal Move {id_from}, {to_dia}. {sol}")
  else:
    new_sol[to_dia].add(id_from)
  new_sol = [dia for dia in new_sol if dia]
  return new_sol

def merge_days(sol: List[Set[int]], iddia0: int, iddia1: int, restriccion: int) -> List[Set[int]]:
  new_sol = [set(dia) for dia in sol]
  new_sol[iddia0].update(new_sol[iddia1])
  if len(new_sol[iddia0]) > restriccion:
    raise Exception("Illegal Move")
  _ = new_sol.pop(iddia1)
  return new_sol

def swap_two_random_between_days(sol: List[Set[int]], i0: int, i1: int) -> List[Set[int]]:
  new_sol = [set(dia) for dia in sol]
  leaving0 = random.sample(new_sol[i0], 2)
  leaving1 = random.sample(new_sol[i1], 2)
  new_sol[i0].difference_update(leaving0)
  new_sol[i1].difference_update(leaving1)
  new_sol[i0].update(leaving1)
  new_sol[i1].update(leaving0)
  return new_sol

MAX_TRIES_GEN_VECINA = 15

def _gen_vecina_aleatorio_swap_tomas(solucion: List[Set[int]], tomas: List[Set[int]]) -> List[Set[int]]:
  i = j = 0
  iter = 0
  check_same_day = True
  if len([dia for dia in solucion if len(dia) > 0]) < 2: # Menos de dos dias no vacios
    check_same_day = False
  while j == i or (check_same_day and [dia for dia in solucion if i in dia and j in dia]):
    i, j = random.sample(range(len(tomas)), 2)
    iter += 1
    if iter > MAX_TRIES_GEN_VECINA:
      return None
  return swap_tomas(solucion, i, j)

def _gen_vecina_aleatorio_move_to(solucion: List[Set[int]], tomas: List[Set[int]], restriccion: int) -> List[Set[int]]:
  i = random.randint(0, len(tomas)-1)
  j = random.randint(0, len(solucion))
  iter = 0
  while j < len(solucion) and (i in solucion[j] or len(solucion[j]) == restriccion):
    i = random.randint(0, len(tomas)-1)
    j = random.randint(0, len(solucion))
    iter += 1
    if iter > MAX_TRIES_GEN_VECINA:
      return None
  return move_to(solucion, i, j, restriccion)

def _gen_vecina_aleatorio_merge_days(solucion: List[Set[int]], tomas: List[Set[int]], restriccion: int) -> List[Set[int]]:
  i = j = 0
  iter = 0
  while j == i or len(solucion[i])+len(solucion[j]) > restriccion:
    i, j = random.sample(range(0, len(solucion)), 2)
    iter += 1
    if iter > MAX_TRIES_GEN_VECINA:
      return None
  return merge_days(solucion, i, j, restriccion)

def _gen_vecina_aleatorio_swap_dos(solucion: List[Set[int]], tomas: List[Set[int]]) -> List[Set[int]]:
  valid_day_indexes = [i for i, day in enumerate(solucion) if len(day) > 3]
  if len(valid_day_indexes) < 2:
    return None
  i = j = 0
  iter = 0
  while j == i:
    i, j = random.sample(valid_day_indexes, 2)
    iter += 1
    if iter > MAX_TRIES_GEN_VECINA:
      return None
  return swap_two_random_between_days(solucion, i, j)

def genera_vecina_aleatorio_improved(solucion: List[Set[int]], tomas: List[Set[int]], restriccion: int):
  each_entorno = 10
  vecinas = [_gen_vecina_aleatorio_swap_tomas(solucion, tomas) for _ in range(each_entorno)]
  vecinas += [_gen_vecina_aleatorio_move_to(solucion, tomas, restriccion) for _ in range(each_entorno)]
  vecinas += [_gen_vecina_aleatorio_merge_days(solucion, tomas, restriccion) for _ in range(each_entorno)]
  vecinas += [_gen_vecina_aleatorio_swap_dos(solucion, tomas) for _ in range(each_entorno)]
  vecinas = [vec for vec in vecinas if vec is not None]
  # Devuelve la mejor
  min_coste = float("inf")
  min_sol = solucion
  for vecina in vecinas:
    coste = coste_actores(vecina, tomas)
    if coste < min_coste:
      min_coste = coste
      min_sol = vecina
  return min_sol


#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(temp: float, d) -> bool:
  return random.random() <  math.exp(-1*d / temp)

#Funcion de descenso de temperatura
def bajar_temperatura(temp: float) -> float:
  return temp * 0.99


def tomas_actores_recocido_simulado(tomas: List[Set[int]], restriccion: int = 6, temp: float = 1000000) -> List[Set[int]]:
  solucion_referencia = tomas_actores_greedy_sinergias(tomas, restriccion)
  distancia_referencia = coste_actores(solucion_referencia, tomas)
  mejor_solucion = solucion_referencia
  mejor_distancia = distancia_referencia
  while temp > .0001:
    vecina = genera_vecina_aleatorio_improved(solucion_referencia, tomas, restriccion)
    distancia_vecina = coste_actores(vecina, tomas)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    if distancia_vecina < distancia_referencia or probabilidad(temp, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    temp = bajar_temperatura(temp)
  return mejor_solucion

solrandom = tomas_actores_recocido_simulado(test_tomas, 3)
print_solucion(solrandom)
print(f"Coste: {coste_actores(solrandom, test_tomas)}")

solr = tomas_actores_recocido_simulado(TOMAS_ENUNCIADO, 6)
print_solucion(solr)
print(f"Coste: {coste_actores(solr, TOMAS_ENUNCIADO)}")

Tomas: Día 1: 1, 3, 6. Día 2: 2, 4, 8. Día 3: 5, 7. 
Coste: 9
Tomas: Día 1: 21, 8, 10, 11, 29, 26. Día 2: 3, 7, 25, 27, 13, 16. Día 3: 17, 18, 19, 23, 24, 14. Día 4: 4, 5, 9, 28, 30, 15. Día 5: 1, 2, 20, 22, 6, 12. 
Coste: 27


## Complejidad de algoritmo mejorado

(*)Calcula la complejidad del algoritmo 

Respuesta

Para calcular la complejidad del algoritmo necesito calcular la complejidad de las diferentes tareas que se realizan en él.

### Algoritmo voraz

Primero el algoritmo voraz, el cual primero calcula las sinergías, lo cual es una función de orden $ O(n²) $, siendo n el número de tomas. Después se realiza un bucle while doble, cuya complejidad dependerá de los datos del problema, pero en el peor caso ambos bucles se ejecutarían n veces, y dentro del bucle exterior se realiza una ordenación, lo que hace que la complejidad de esa sección sea aproximadamente $n*(n+nlogn)$ que equivale a $O(n²logn)$.
Finalmente, en el algoritmo voraz se realiza una compactación simple de los días de orden $O(n²)$.

Todo esto significa que la complejidad total del algoritmo voraz es $O(n²logn)$.

### Recocido simulado

Tanto cada función para la generación de vecinas como la función de cálculo de coste son de coste $ O(m) $ siendo m el número de días de la solución. Como el número de días es variable, debería darse en función del número de tomas. Como el número de días va a estar entre n y n/k, siendo k la restricción, puede aproximarse el coste a $ O(n) $.

El algoritmo de recocido simulado itera en función de la temperatura, T. El valor T se va multiplicando por 0.99 hasta que alcanza 0.001. Esto significa que se ejecuta $log$<sub>0.99</sub>$0.0001\over T$ veces. Se puede inferir fácilmente que el orden de complejidad es de $O(nlog$$1\over T$$)$

### Coste total

El coste total es la suma de ambos costes, es decir:
$$ O(n²logn) + O(nlog(1/T)) = O(n²logn)$$

Se puede ver que matemáticamente el coste del recocido simulado afecta a la escalabilidad en menor medida que el algoritmo voraz, sin embargo he comprobado de forma empírica que tiene un coste temporal constante que provoca que tarde más que la parte del algoritmo voraz en muchos casos, como para el caso del enunciado.

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [9]:
n_actores = random.randint(5,20)
n_tomas = random.randint(n_actores,n_actores+30)
print(f"Nº Actores: {n_actores}. Nº Tomas: {n_tomas}.")
tomas_aleatorias = [{*random.sample(range(n_actores), random.randint(1, n_actores))} for t in range(n_tomas)]
display(tomas_aleatorias)

Nº Actores: 12. Nº Tomas: 31.


[{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
 {0, 2, 8, 10},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
 {3, 5, 6, 8, 9},
 {1, 2, 4, 6, 7, 8, 9, 10, 11},
 {0, 1, 2, 5, 6, 7, 8, 10, 11},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10},
 {6},
 {0, 1, 2, 4, 5, 7, 8, 9, 10, 11},
 {0, 2, 3, 4, 5, 7, 8, 9, 10},
 {0, 2, 4, 5, 8, 9, 10, 11},
 {3},
 {0, 1, 3, 6, 7, 8, 9, 10, 11},
 {0, 1, 2, 4, 5, 6, 7, 8, 9, 10},
 {0, 1, 2, 3, 7, 9},
 {1, 4, 6, 7, 8, 10},
 {0, 1, 3, 4, 6, 9},
 {1, 2, 5, 7, 8, 9, 11},
 {5, 7, 8, 9, 11},
 {0, 2, 3, 4, 5, 6, 7, 11},
 {2, 6, 7, 10},
 {1, 10},
 {6, 8, 11},
 {0, 1, 5, 6, 11},
 {0, 1, 2, 4, 6, 9, 10},
 {1, 2, 3, 4, 6, 8, 10, 11},
 {0, 1, 2, 4, 5, 6, 7, 10, 11},
 {5, 6, 9},
 {1, 3, 4, 5, 6, 7, 8, 9, 10, 11},
 {1, 2},
 {0, 3, 4, 5, 6, 7, 8, 9, 10, 11}]

## Aplica el algoritmo al juego de datos generado

Respuesta

In [10]:
sol_datos = tomas_actores_recocido_simulado(tomas_aleatorias, 6)
print_solucion(sol_datos)
print(f"Coste algoritmo recocido simulado: {coste_actores(sol_datos, tomas_aleatorias)}")
solran = random_solution(tomas_aleatorias, 6)
print_solucion(solran)
print(f"Coste solución aleatoria: {coste_actores(solran, tomas_aleatorias)}")
solgreedy = tomas_actores_greedy_sinergias(tomas_aleatorias, 6)
print_solucion(solgreedy)
print(f"Coste algoritmo voraz: {coste_actores(solgreedy, tomas_aleatorias)}")

Tomas: Día 1: 3, 20, 7, 9, 10, 15. Día 2: 18, 19, 5, 25, 11, 14. Día 3: 1, 17, 26, 13, 29, 31. Día 4: 2, 21, 6, 24, 27, 16. Día 5: 30, 22. Día 6: 4, 23, 8, 28, 12. 
Coste algoritmo recocido simulado: 54
Tomas: Día 1: 11, 29. Día 2: 3, 8, 9, 19, 20, 23. Día 3: 15. Día 4: 25, 17, 7. Día 5: 10, 27, 5, 16. Día 6: 1, 6, 13, 14. Día 7: 22, 31. Día 8: 12, 28, 30, 24. Día 9: 2, 4, 21, 18. Día 10: 26. 
Coste solución aleatoria: 103
Tomas: Día 1: 1, 3, 7, 9, 29, 31. Día 2: 5, 6, 10, 11, 14, 27. Día 3: 13, 15, 16, 17, 18, 26. Día 4: 2, 12, 19, 20, 21, 24. Día 5: 25, 30, 22, 8. Día 6: 28, 4, 23. 
Coste algoritmo voraz: 61


## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Además de los contenidos propios de la asignatura (Transparencias, actividades guiadas...) he utilizado las siguientes referencias:


Números de Stirling de segundo tipo y números de Bell: https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind

Números de Stirling de segundo tipo restringidos: https://math.stackexchange.com/questions/2302204/counting-set-partitions-of-1-2-n-into-exactly-k-non-empty-subsets-wi

Números de Bell restringidos: https://math.stackexchange.com/questions/3685565/set-partitioning-with-max-partition-size y https://arxiv.org/pdf/1707.08138.pdf

## Posibles avances en el problema

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Considero que el algoritmo voraz puede ser mejorado, ya que el problema podría someterse a un estudio mucho más prolongado, comparando con muchos más datos de entrada, y posiblemente encontrar un algoritmo voraz similar que de mejores resultados. En la elaboración del algoritmo voraz me centré en que los resultados mejoraran para los datos del enunciado, pero seguramente haya otros datos de entrada posible para los que pudiera ser fácilmente optimizado.

La parte del recocido simulado también podría ser mejorada. Sólo obtengo una mejora de dos puntos para la entrada del enunciado, y quizá sea posible mejorar más. Los espacios de vecindad planteados creo que son correctos, pero quizá se podrían generar vecinos mejores de forma inteligente en vez de aleatoria, por ejemplo utilizando la tabla de sinergías. También creo que quizá podría mejorarse utilizando una lista tabú evitando volver a soluciones anteriores que dieron mejor resultado, para así evitar estancarse en mínimos locales, que creo que puede ser un problema que tenga.

También podría ser interesante diseñar un algoritmo voraz más simplificado y una generación de vecinas más simple, pues en caso de que se aumente muchísimo el tamaño (por ejemplo millones de tomas y millones de actores), probablemente mi algoritmo dejara de ser factible.